In [ ]:
import ssl
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context # Only used if pandas gives a SSLError

def fetch(exchange_name, symbol, timeframe):
    #url = "https://www.cryptodatadownload.com/cdd/"
    url = './data/'
    filename = "{}_{}USD_{}.csv".format(exchange_name, symbol, timeframe)
    volume_column = "Volume {}".format(symbol)
    new_volume_column = "Volume_{}".format(symbol)
    
    df = pd.read_csv(url + filename, skiprows=1)
    df = df[::-1]
    df = df.drop(["Symbol"], axis=1)
    df = df.rename({"Volume USD": "volume", volume_column: new_volume_column}, axis=1)
    df = df.set_index("Date")
    df.columns = [symbol + ":" + name.lower() for name in df.columns]
                     
    return df

## Setup Trading Environment

In [ ]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream, DataFeed, Module
from tensortrade.instruments import USD, BTC, ETH, LTC
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.environments import TradingEnvironment

# a dataframe
coinbase_data = pd.concat([
    fetch("Coinbase", "BTC", "1h"),
    fetch("Coinbase", "ETH", "1h")
], axis=1)

# an exchange instance with `execute_order` service and two Stream input (Call __call__ funtion of class Exchange inherit from class Node with two Stream instance params)
coinbase = Exchange("coinbase", service=execute_order)(
    Stream("USD-BTC", list(coinbase_data['BTC:close'])),
    Stream("USD-ETH", list(coinbase_data['ETH:close']))
)

# a dataframe of btc data
coinbase_btc = coinbase_data.loc[:, [name.startswith("BTC") for name in coinbase_data.columns]]

# Module coinbase_ns with several subnodes(varibales) of Stream instance
with Module("coinbase") as coinbase_ns:
    nodes = [Stream(name, list(coinbase_data[name])) for name in coinbase_data.columns]

# datafeed which inputs includes all subnodes(variables) of Module coinbase_ns
# feed.next() will return the next value of all inputs of the DataFeed
# feed.reset() will reset all inputs of the DataFeed
feed = DataFeed([coinbase_ns])

# portfolio with base instrument USD and three Wallet of different Exchange and Quantity
portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC),
    Wallet(coinbase, 5 * ETH),
])

# TradingEnviroment contains all components used in trading
# in compile method of the env, it calculates the size of action spaces, merge outernal and internal Data 
# feed
env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    action_scheme='managed-risk',
    reward_scheme='risk-adjusted',
    window_size=100
)

## Example Data Feed Observation

In [ ]:
env.feed.next()

## Setup and Train DQN Agent

In [ ]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)

agent.train(n_episodes=10, save_path="./agents/")

## Plot Performance

In [ ]:
%matplotlib inline

portfolio.performance.plot()

In [ ]:
portfolio.performance.net_worth.plot()

## Setup and Train A2C Agent

In [ ]:
from tensortrade.agents import A2CAgent

agent = A2CAgent(env)

agent.train(n_episodes=10, save_path="./agents/")

In [ ]:
portfolio.performance.plot()

In [ ]:
portfolio.performance.net_worth.plot()

## Setup and Train Parallel DQN Agent

In [ ]:
from tensortrade.agents import ParallelDQNAgent

def create_env():
    feed = DataFeed([coinbase_ns])

    portfolio = Portfolio(USD, [
        Wallet(coinbase, 10000 * USD),
        Wallet(coinbase, 10 * BTC),
        Wallet(coinbase, 5 * ETH),
    ])
    
    env = TradingEnvironment(
        feed=feed,
        portfolio=portfolio,
        action_scheme='managed-risk',
        reward_scheme='risk-adjusted',
        window_size=100
    )
    
    return env

agent = ParallelDQNAgent(create_env)

agent.train(n_envs=4, n_episodes=10, save_path="./agents/")

In [ ]:
portfolio.performance.plot()

In [ ]:
portfolio.performance.net_worth.plot()